# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116141e+02     1.489944e+00
 * time: 0.3553040027618408
     1     1.112370e+01     1.036767e+00
 * time: 2.339387893676758
     2    -1.166982e+01     1.141250e+00
 * time: 2.990356922149658
     3    -3.384333e+01     9.442593e-01
 * time: 3.957828998565674
     4    -4.735641e+01     7.050693e-01
 * time: 5.259008884429932
     5    -5.686075e+01     2.241356e-01
 * time: 6.209836006164551
     6    -5.978611e+01     1.497310e-01
 * time: 6.854121923446655
     7    -6.087141e+01     5.330224e-02
 * time: 7.495675802230835
     8    -6.126668e+01     7.281446e-02
 * time: 8.349416017532349
     9    -6.152443e+01     3.755492e-02
 * time: 8.991102933883667
    10    -6.172425e+01     2.802482e-02
 * time: 9.640851020812988
    11    -6.188987e+01     2.477482e-02
 * time: 10.280157804489136
    12    -6.196118e+01     1.908193e-02
 * time: 10.92098879814148
    13    -6.203820e+01     1.706141e-02
 * time: 11.572573900222778
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671079
    AtomicLocal         -18.8557700
    AtomicNonlocal      14.8522648
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485387 
    Xc                  -19.3336827

    total               -62.261666460909
